In [2]:
import os
import pandas as pd
import numpy as np

cols = [2,3,4,7,8,9]

PubChemDF = pd.DataFrame()
PCBA_ID = []

for filename in os.listdir(os.getcwd()):
    if filename.startswith('AID'):
        assay_id = filename.rsplit('_')[1]
        df = pd.read_csv(filename, skiprows = list(range(1,6)))#Skip rows with column description
        #df = df.iloc[:,2:5] #Select needed columns
        df = df.iloc[:,cols]
        df.columns = ['CID', assay_id + '_OUTCOME', assay_id + '_SCORE', assay_id + 'PHENOTYPE', assay_id + '_POTENCY', assay_id + '_EFFICACY']
        PubChemDF = pd.concat([PubChemDF, df]).groupby('CID', as_index=False, sort=False).first().fillna('NA')
        PCBA_ID.append(assay_id)

/ihome/gidakwo/anaconda3/envs/ml_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
import pubchempy as pcp
from rdkit import Chem
import molvs as mv

def get_parent_smile(cid):
    try:
        cpd = pcp.Compound.from_cid(int(cid))
        smi = cpd.isomeric_smiles
        st = mv.Standardizer()
        mol = Chem.MolFromSmiles(smi)
        smts = Chem.MolFromSmarts("[!#1&!#5&!#6&!#7&!#8&!#9&!#14&!#15&!#16&!#17&!#34&!#35&!#53]~*")
        if mol.HasSubstructMatch(smts) == False:
            mols = st.charge_parent(mol)
            return Chem.MolToSmiles(mols)
    except:
        return 'NaN'

#Generate InchiKey
def inchikey_gen(smile):
    try:
        m = Chem.MolFromSmiles(smile)
        m_ = Chem.MolToInchi(m)
        m__ = Chem.InchiToInchiKey(m_)
        return m__
    except:
        return 'NaN'
    
#Clean and standardize the data
def clean_data(data):
    #Standardize and get parent with molvs
    data["smiles_parent"] = data['CID'].apply(get_parent_smile)
    #data = data[data['smiles_parent'].notnull()]
    data = data[~(data['smiles_parent'] == "NaN")]
    
    data["inchi"] = data['smiles_parent'].apply(inchikey_gen)
    data = data[~(data['inchi'] == "NaN")]
    #data = data.drop('CID', axis = 1)
    return data

In [6]:
PubChemDF = clean_data(PubChemDF)

/ihome/gidakwo/anaconda3/envs/ml_env/lib/python3.5/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
PubChemDF.to_csv('PubChemDF_new.csv')